# Assignment 3 Dashboard
**Name: Rommel Antunez Barrios**

**e-mail: rommel.antunez6474@alumnos.udg.mx**

# Modules

In [10]:
import numpy as np
from scipy.stats import wrapcauchy
from matplotlib import pyplot as plt
from scipy.spatial import distance
import math

from scipy.stats import levy_stable

# from plotly
import plotly.graph_objects as go

# from pandas
import pandas as pd

# from panel
import panel as pn
pn.extension() # this is needed to use plotly in panel

# Functions and classes

## Vec2d

In [11]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## Brownian Motion

In [12]:
def brownian_motion(speed=3, n_steps=1000, initial_x=0, initial_y=0):
    # Init velocity vector
    velocity = Vec2d(speed,0)

    # Init empty DF
    BM_2d_df = pd.DataFrame(columns=['x_pos','y_pos', 'z_pos'])
    # Add starting position to DF
    temp_df = pd.DataFrame([{'x_pos':initial_x, 'y_pos':initial_y, 'z_pos':0}])

    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True) 
    # Generate trajectory
    for i in range(n_steps-1):
        turn_angle = np.random.choice([0, np.pi/2, np.pi, 3*np.pi/2])
        # We are rotating the velocity vector
        velocity = velocity.rotated(turn_angle)

        # Update to new position
        temp_df = pd.DataFrame([{
                'x_pos':BM_2d_df.x_pos[i] + velocity.x, 
                'y_pos':BM_2d_df.y_pos[i] + velocity.y,
                'z_pos':BM_2d_df.z_pos[i] + velocity.get_length()
            }])
        BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True) 
    return BM_2d_df

## Levy Flight

In [42]:
def levy_flight(speed=5, n_steps=1000, initial_x=0, initial_y=0,cauchy_coefficient=1.2):
    # Initial variables
    time_per_step=0.0001
    max_step_length=10
    s_pos = [initial_x, initial_y, 0]
    beta = 0
    m = 1

    trajectory = [{'x_pos': s_pos[0], 'y_pos': s_pos[1], 'z_pos': s_pos[2]}]

    # Implementation of Vec2d class
    velocity = Vec2d(speed, 0)

    rotations = levy_stable.rvs(cauchy_coefficient, beta, loc=m, scale=1, size=n_steps)

    for i in range(n_steps):
        turn_angle = rotations[i]  # Get rotations from levy distribution.
        step_length = levy_stable.rvs(cauchy_coefficient, beta)
        step_length = min(step_length, max_step_length)
        velocity = velocity.rotated(turn_angle)  # Use rotated function from Vec2d class

        # Dict, later to be appended to list
        new_position = {
            'x_pos': trajectory[i]['x_pos'] + velocity.x * float(step_length),
            'y_pos': trajectory[i]['y_pos'] + velocity.y * float(step_length),
            'z_pos': trajectory[i]['z_pos'] + velocity.get_length() * time_per_step * float(step_length)  # Calculated distance
        }
        # List with all the trajectory position
        trajectory.append(new_position)

    # Create Pandas Data Frame with trajectories
    levy_pdf = pd.DataFrame(trajectory)
    return levy_pdf


# Dashboard

In [43]:
# ----------------- CREATE WIDGETS -----------------


# Create radio buttons for filter selection
trajectory_filter_radio = pn.widgets.RadioButtonGroup(
    name='Filter', 
    options=['BM', 'CRW', 'LF'],
    value='BM'
)

metric_filter_radio = pn.widgets.RadioButtonGroup(
    name='Metric',
    options=['PL', 'MSD', 'TAD'],
    value='PL'
)

# Create widgets for number of steps, speed, and initial positions
steps_slider = pn.widgets.IntSlider(name='Number of Steps', start=50, end=1000, step=50, value=50)
speed_slider = pn.widgets.IntSlider(name='Speed', start=1, end=12, step=1, value=1)
init_x_input = pn.widgets.IntInput(name='Init X Position', value=0)
init_y_input = pn.widgets.IntInput(name='Init Y Position', value=0)

# Create widget for cauchy coefficient, initially hidden
cauchy_coeff_input = pn.widgets.FloatInput(name='Cauchy Coefficient', value=1.2, visible=False)

# ----------------- FUNCTION -----------------

# Function to show/hide cauchy coefficient input based on filter selection
def update_cauchy_coeff(event):
    if trajectory_filter_radio.value in ['CRW', 'LF']:
        cauchy_coeff_input.visible = True
    else:
        cauchy_coeff_input.visible = False

# Attach the update function to the filter radio button group
trajectory_filter_radio.param.watch(update_cauchy_coeff, 'value')

# Plot Different Trajectories
@pn.depends(steps_slider, speed_slider, init_x_input, init_y_input, cauchy_coeff_input, trajectory_filter_radio)
def plot_traj(steps_slider, speed_slider, init_x_input, init_y_input, cauchy_coeff_input, trajectory_filter_radio):
    if trajectory_filter_radio == 'BM':
        # Generate Brownian Motion
        trajectory = brownian_motion(speed=speed_slider, n_steps=steps_slider, initial_x=init_x_input, initial_y=init_y_input)
        plot_name='Brownian Motion'
    elif trajectory_filter_radio == 'LF':
        # Generate Levy Flight
        trajectory = levy_flight(speed=speed_slider, n_steps=steps_slider, initial_x=init_x_input, initial_y=init_y_input, cauchy_coefficient=cauchy_coeff_input)
        plot_name='Levy Flight'
    else:
        return None
    
    # Plot the trajectory
    fig_3d = go.Figure()
    fig_3d.add_trace(
        go.Scatter3d(
            x = trajectory.x_pos,
            y = trajectory.y_pos,
            z = trajectory.z_pos,
            mode = 'lines',
            line=dict(color='green', width=1), 
            name = plot_name,
            showlegend = True
        )
    )
    
    fig_3d.update_yaxes(scaleanchor='x', scaleratio=1)
    return fig_3d

# ----------------- CREATE DASHBOARD -----------------

# Create the layout for the first column
column1 = pn.Column(
    trajectory_filter_radio,
    steps_slider,
    speed_slider,
    init_x_input,
    init_y_input,
    cauchy_coeff_input,
    metric_filter_radio
)

# Plot(3D plot)
column2 = pn.pane.Plotly(plot_traj)

# Placeholder for the third column (Graphic)
column3 = pn.pane.Markdown("Here will be the metric graphic")

# Create the main layout with three columns
main_layout = pn.Row(column1, column2 , column3)

# Display the panel
main_layout.servable()



BokehModel(combine_events=True, render_bundle={'docs_json': {'e33363d3-b998-455e-bc6d-47edeb10f3a7': {'version…

-0.9976762071431969
<class 'numpy.ndarray'>
-0.1455734338667862
<class 'numpy.ndarray'>
-0.9789635750027755
<class 'numpy.ndarray'>
5.1655650832698194
<class 'numpy.ndarray'>
-1.0282652282882232
<class 'numpy.ndarray'>
0.031011595893767424
<class 'numpy.ndarray'>
0.8995627306906115
<class 'numpy.ndarray'>
-0.29385953799363307
<class 'numpy.ndarray'>
-1.2111881930825381
<class 'numpy.ndarray'>
0.2874789299398794
<class 'numpy.ndarray'>
-1.0257167015210396
<class 'numpy.ndarray'>
0.2659485731787776
<class 'numpy.ndarray'>
1.5786903232592469
<class 'numpy.ndarray'>
0.4010321546573874
<class 'numpy.ndarray'>
1.2489328932043076
<class 'numpy.ndarray'>
7.085746420222418
<class 'numpy.ndarray'>
0.6082146928021402
<class 'numpy.ndarray'>
-0.09300387333117914
<class 'numpy.ndarray'>
1.1436730561216166
<class 'numpy.ndarray'>
-4.4042083993613295
<class 'numpy.ndarray'>
3.500058110802666
<class 'numpy.ndarray'>
-0.7572306899727502
<class 'numpy.ndarray'>
3.1124653143729555
<class 'numpy.ndarray'>